In [ ]:
from finance.data.db.mysql import MySQLClient
from finance.data.fundamentals import upsert_fundamentals
from finance.data.factors import  upsert_factors
from finance.data.asset_profile import load_symbols_from_asset_profile
from edgar import Company, set_identity

import pandas as pd


[
    'Total Revenue',                                # 총매출 (회사 전체의 매출액)
    'Cost of Revenue',                              # 매출원가 (제품/서비스 판매에 소요된 비용)
    'Gross Profit',                                 # 매출총이익 (총매출 - 매출원가)
    'Operating expenses:',                          # 영업비용 (운영에 소요된 비용)
    'Research and Development Expense',             # 연구개발비 (신제품/서비스 개발비용)
    'Selling and Marketing Expense',                # 판매 및 마케팅 비용
    'General and Administrative Expense',           # 일반관리비 (경영, 행정 비용)
    'Operating Expenses',                           # 영업비용 전체
    'Selling, General and Administrative Expense',  # 판매 관리비 (SG&A)
    'Operating Income (Loss)',                      # 영업이익 (손실)
    'Other Nonoperating Income (Expense)',          # 기타 영업외 수익(비용)
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest',  # 계속영업이익 (법인세 차감 전, 비지배 지분 전)
    'Income Tax Expense (Benefit)',                 # 법인세 비용(혜택)
    'Net Income (Loss) Attributable to Parent',     # 순이익 (지배기업 귀속)
    'Earnings Per Share, Basic',                    # 주당 순이익 (기본)
    'Earnings Per Share, Diluted',                  # 주당 순이익 (희석)
    'Other income (expense):',                      # 기타수익(비용)
    'Nonoperating Income (Expense)',                # 영업외 수익(비용)
    'Additional Financial Items',                   # 추가 재무 항목
    'Amortization of Intangible Assets',            # 무형자산상각비
    'Common Stock, Dividends, Per Share, Declared', # 보통주 1주당 배당금
    'Depreciation, Depletion and Amortization, Nonproduction',  # 감가상각비(비생산활동 관련)
    'Goodwill, Impairment Loss',                    # 영업권 손상차손
    'Interest Expense',                             # 이자 비용
    'Interest Expense, Debt',                       # 부채 이자 비용
    'Reclassification from AOCI, Current Period, before Tax, Attributable to Parent', # 기타포괄손익누계액의 재분류 (지배기업 귀속)
    'Revenue, Net (Deprecated 2018-01-31)'          # 순매출(구버전, 더 이상 사용안함)
]

financial_term_kor_map = {
    'Total Revenue': '총매출 (회사 전체의 매출액)',
    'Cost of Revenue': '매출원가 (제품/서비스 판매에 소요된 비용)',
    'Gross Profit': '매출총이익 (총매출 - 매출원가)',
    'Operating expenses:': '영업비용 (운영에 소요된 비용)',
    'Research and Development Expense': '연구개발비 (신제품/서비스 개발비용)',
    'Selling and Marketing Expense': '판매 및 마케팅 비용',
    'General and Administrative Expense': '일반관리비 (경영, 행정 비용)',
    'Operating Expenses': '영업비용 전체',
    'Selling, General and Administrative Expense': '판매 관리비 (SG&A)',
    'Operating Income (Loss)': '영업이익 (손실)',
    'Other Nonoperating Income (Expense)': '기타 영업외 수익(비용)',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': '계속영업이익 (법인세 차감 전, 비지배 지분 전)',
    'Income Tax Expense (Benefit)': '법인세 비용(혜택)',
    'Net Income (Loss) Attributable to Parent': '순이익 (지배기업 귀속)',
    'Earnings Per Share, Basic': '주당 순이익 (기본)',
    'Earnings Per Share, Diluted': '주당 순이익 (희석)',
    'Other income (expense):': '기타수익(비용)',
    'Nonoperating Income (Expense)': '영업외 수익(비용)',
    'Additional Financial Items': '추가 재무 항목',
    'Amortization of Intangible Assets': '무형자산상각비',
    'Common Stock, Dividends, Per Share, Declared': '보통주 1주당 배당금',
    'Depreciation, Depletion and Amortization, Nonproduction': '감가상각비(비생산활동 관련)',
    'Goodwill, Impairment Loss': '영업권 손상차손',
    'Interest Expense': '이자 비용',
    'Interest Expense, Debt': '부채 이자 비용',
    'Reclassification from AOCI, Current Period, before Tax, Attributable to Parent': '기타포괄손익누계액의 재분류 (지배기업 귀속)',
    'Revenue, Net (Deprecated 2018-01-31)': '순매출(구버전, 더 이상 사용안함)',
    'Business Combination, Acquisition Related Costs': '사업결합, 인수 관련 비용',
    'Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization': '감가상각/고갈/무형자산상각 제외 상품 및 서비스 원가',
    'Cost of Goods Sold (Deprecated 2018-01-31)': '매출원가(구버전, 더 이상 사용안함)',
    'Current Income Tax Expense (Benefit)': '당기 법인세 비용(혜택)',
    'Equity Securities without Readily Determinable Fair Value, Impairment Loss, Annual Amount': '공정가치 산정 불가 지분증권 손상차손 (연간)',
    'Income (Loss) from Continuing Operations, Net of Tax, Attributable to Parent': '계속영업이익(세후, 지배기업 귀속)',
    'Income (Loss) from Continuing Operations before Income Taxes, Domestic': '국내 계속영업이익 (법인세 이전)',
    'Income (Loss) from Continuing Operations, Per Basic Share': '계속영업 주당 순이익(기본)',
    'Income (Loss) from Continuing Operations, Per Diluted Share': '계속영업 주당 순이익(희석)',
    'Income (Loss) from Discontinued Operations, Net of Tax, Attributable to Parent': '중단영업이익(손실, 세후, 지배기업 귀속)',
    'Income (Loss) from Equity Method Investments': '지분법 투자이익(손실)',
    'Interest Income (Expense), Net': '이자수익(비용) 순액',
    'Marketing and Advertising Expense': '마케팅 및 광고 비용',
    'Operating Lease, Expense': '운영리스 비용',
    'Restructuring and Related Cost, Incurred Cost': '구조조정 및 관련 비용, 확정된 비용',
    'Restructuring Costs': '구조조정 비용',
    'Sales Revenue, Goods, Net (Deprecated 2018-01-31)': '상품 순매출 (구버전, 더 이상 사용안함)',
    'Net Income (Loss) Attributable to Noncontrolling Interest': '순이익(비지배지분 귀속)',
    'Investment Income, Interest': '이자성 투자수익',
    'Foreign Currency Transaction Gain (Loss), before Tax': '외화거래이익(손실, 세전)',
    'Foreign Currency Transaction Gain (Loss), Realized': '외화거래이익(손실, 실현)',
    'Gain (Loss) on Investments': '투자이익(손실)',
    'Net Income (Loss) Available to Common Stockholders, Basic': '보통주주 귀속 순이익(손실, 기본)',
    'Expenses': '비용',
    'Policyholder Benefits and Claims Incurred, Net': '보험계약자 지급 및 발생손해, 순액',
    'Deferred Policy Acquisition Costs, Amortization Expense': '이연 보험계약 인수비용 상각',
    'Benefits, Losses and Expenses': '보험급여, 손실 및 비용',
    'Other Cost and Expense, Operating': '기타 운영비용',
    'Capitalized Computer Software, Amortization': '자본화된 소프트웨어 상각',
    'Equity Securities, FV-NI, Realized Gain (Loss)': '공정가치-당기손익 지분증권, 실현손익',
    'Investment Income, Net': '순투자수익',
    'Liability for Unpaid Claims and Claims Adjustment Expense, Incurred Claims': '미지급 보험금 및 청구조정비용 발생손해',
    'Other Expenses': '기타 비용',
    'Preferred Stock Dividends, Income Statement Impact': '우선주 배당금, 손익계산서 영향'
}

set_identity("honeypipeline@gmail.com")

def get_fundamental(symbol:str, periods=4, period="annual")->pd.DataFrame:
    c = Company(symbol)
    income = c.income_statement(periods=periods, period=period, as_dataframe=True)
    income = income.reset_index()
    income = income.drop(columns=['concept', 'depth', 'is_abstract', 'is_total','section'])

    fy_cols = income.filter(regex=r"^FY").columns   # FY로 시작하는 컬럼들(늘어나도 자동)
    income = income.dropna(subset=fy_cols, how="all")

    income['label_kr'] = income['label'].map(financial_term_kor_map)
    income['symbol'] = symbol

    col = income.pop("label_kr")
    i = income.columns.get_loc('label') + 1
    income.insert(i, "label_kr", col)

    col = income.pop("symbol")
    income.insert(0,"symbol", col)

    return income


labels = []
symbols = ['AAME']
keys = set(financial_term_kor_map.keys())
for s in symbols:
    df = get_fundamental(s)
    display(df)






,symbol,label,label_kr,confidence,FY 2024,FY 2023,FY 2022,FY 2021
1,AAME,Assets,NaN,0.900302,393428000.0,381265000.0,367064000.0,402286000.0
3,AAME,"Cash and Cash Equivalents, at Carrying Value",NaN,0.728097,35570000.0,28301000.0,28863000.0,24753000.0
4,AAME,"Operating Lease, Right-of-Use Asset",NaN,0.459215,5225000.0,2614000.0,3405000.0,4143000.0
6,AAME,Liabilities,NaN,0.728097,293815000.0,273990000.0,264871000.0,261000000.0
8,AAME,"Preferred Stock, Value, Issued",NaN,0.416918,55000.0,55000.0,55000.0,55000.0
9,AAME,"Common Stock, Value, Issued",NaN,0.770393,22401000.0,22401000.0,22401000.0,22401000.0
10,AAME,Additional Paid in Capital,NaN,0.447130,57425000.0,57425000.0,57425000.0,57441000.0
11,AAME,Retained Earnings (Accumulated Deficit),NaN,0.836858,45854000.0,50929000.0,51982000.0,51264000.0
12,AAME,"Accumulated Other Comprehensive Income (Loss),...",NaN,0.755287,-18712000.0,-16121000.0,-22149000.0,17688000.0
13,AAME,Stockholders' Equity Attributable to Parent,NaN,0.815710,99613000.0,107275000.0,102193000.0,141286000.0
